In [1]:
%load_ext micropython_magic

Current ip.InteractiveTB.mode='Context'
New ip.InteractiveTB.mode='Minimal'


In [2]:
%mpy --verify

In [3]:
%mpy import sys;print(f"Testing on {sys.implementation.name} {sys.platform}")

['Testing on micropython pyboard']

In [4]:
!mpy-cross mp_typing_min.py -O3
!mpremote cp mp_typing_min.mpy :typing.mpy


cp mp_typing_min.mpy :typing.mpy


In [14]:
# %%micropython

from typing import Protocol


class Adder(Protocol):
    def add(self, x, y): ...


class IntAdder:
    def add(self, x, y):
        return x + y


class FloatAdder:
    def add(self, x, y):
        return x + y


def add(adder: Adder) -> None:
    print(adder.add(2, 3))


add(IntAdder())
add(FloatAdder())

5
5


In [6]:
# %%micropython

from typing import List, Protocol

l: List[int] = [1, 2, 3]


class Speak(Protocol):
    def speak(self): ...


class Parrot:
    def speak(self) -> None:
        print("Polly wants a cracker")


def say_something(speaker: Speak) -> None:
    speaker.speak()


polly = Parrot()

print(say_something(polly))

Polly wants a cracker
None


In [7]:
# %%python3
from typing import List, Protocol

In [8]:
!mpy-cross mp_abc_min.py -O3
!mpremote cp mp_abc_min.mpy :abc.mpy

cp mp_abc_min.mpy :abc.mpy


In [9]:
# %%micropython

from abc import ABC, abstractmethod
from math import pi


class Shape(ABC):
    @abstractmethod
    def get_area(self) -> float:
        pass

    @abstractmethod
    def get_perimeter(self) -> float:
        pass


class Circle(Shape):
    def __init__(self, radius) -> None:
        self.radius = radius

    def get_area(self) -> float:
        return pi * self.radius**2

    def get_perimeter(self) -> float:
        return 2 * pi * self.radius


class Square(Shape):
    def __init__(self, side) -> None:
        self.side = side

    def get_area(self) -> float:
        return self.side**2

    def get_perimeter(self) -> float:
        return 4 * self.side

In [10]:
# %%micropython

from abc import ABCMeta


# class MyABC(metaclass=ABCMeta):
class MyABC(ABCMeta):
    pass

In [11]:
%ls *_min.mpy

 Volume in drive D is DEV
 Volume Serial Number is E61F-E2CD

 Directory of d:\mypython\!-stubtestprojects\rt_typing

30-08-2024  21:53               116 mp_abc_min.mpy
30-08-2024  21:53               324 mp_typing_min.mpy
               2 File(s)            440 bytes
               0 Dir(s)  470.708.424.704 bytes free


In [12]:
# %%micropython

from abc import ABCMeta


class MyABC(metaclass=ABCMeta):
    pass

WARNING  | Traceback (most recent call last):
WARNING  |   File "<stdin>", line 6, in <module>
ERROR    | TypeError: function doesn't take keyword arguments



MCUException: TypeError: function doesn't take keyword arguments
